<a href="https://colab.research.google.com/github/isegura/iso4simplify/blob/main/select_random_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Escribe los nombres de tus columnas
COL_ID='pair_id'
COL_COMPLEX='complex'   # texto original
COL_REF='simple'        # texto simplificado
# Número de ejemplos a generar
N_EXAMPLES=3
# Nombre del dataset
DATASET='tsar2024'

# Ruta relativa al fichero de training
PATH_TRAIN='data/tsar2024/cochraneauto_docs_train.csv'
# Ruta relativa de la template de prompts
TEMPLATE_PATH = "prompts/template_iso_few.md"


In [2]:
from google.colab import drive
import os, pandas as pd

# montar disco
drive.mount('/content/drive')
# Path del proyecto
PATH='/content/drive/MyDrive/iso4simplify'
os.chdir(PATH)


df = pd.read_csv(PATH_TRAIN)
print("Train size:", df.shape)
#print("Train columns:", df.columns)

df = df[[COL_ID, COL_COMPLEX, COL_REF]]
df_random = df.sample(n=N_EXAMPLES, random_state=42)
#df_random.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train size: (849, 10)


#2) Cargamos el template del prompt


In [3]:
from pathlib import Path

DIR_PROMPTS='prompts'
os.makedirs(DIR_PROMPTS, exist_ok=True)


template_few = Path(TEMPLATE_PATH).read_text(encoding="utf-8")
examples = []

for i, row in df_random.iterrows():
    examples.append({
        "id": row[COL_ID].strip(),
        "input": row[COL_COMPLEX].strip(),
        "output": row[COL_REF].strip()
    })

prompt_fewshot = template_few.format(
    ID_1=examples[0]["id"],
    EXAMPLE_1_INPUT=examples[0]["input"],
    EXAMPLE_1_OUTPUT=examples[0]["output"],

    ID_2 =examples[1]["id"],
    EXAMPLE_2_INPUT=examples[1]["input"],
    EXAMPLE_2_OUTPUT=examples[1]["output"],

    ID_3 =examples[0]["id"],
    EXAMPLE_3_INPUT=examples[2]["input"],
    EXAMPLE_3_OUTPUT=examples[2]["output"],
    INPUT_TEXT="{INPUT_TEXT}"  # placeholder para inferencia
)

OUTPUT_PATH = f"prompts/{DATASET}_random_few_{N_EXAMPLES}.md"

with open(OUTPUT_PATH, "w", encoding="utf-8") as f:
    f.write(prompt_fewshot)

print("Prompt saved to:", OUTPUT_PATH)

Prompt saved to: prompts/tsar2024_random_few_3.md


In [4]:

output_file = DIR_PROMPTS+f"/{DATASET}_random_{N_EXAMPLES}.txt"

with open(output_file, "w", encoding="utf-8") as f:

    for idx, (_, row) in enumerate(df_random.iterrows(), start=1):

        f.write("="*50 + "\n\n")

        f.write(f"EXAMPLE {idx} ({row[COL_ID]})\n")
        f.write("ORIGINAL:\n")
        f.write(str(row[COL_COMPLEX]).strip() + "\n\n")

        f.write("SIMPLIFIED TEXT:\n")
        f.write(str(row[COL_REF]).strip() + "\n\n")

    f.write("="*50 + "\n")

print("Fichero generado en:", os.path.abspath(output_file))

Fichero generado en: /content/drive/MyDrive/iso4simplify/prompts/tsar2024_random_3.txt
